In [ ]:
import json
import logging
import os
from typing import Union

import pandas as pd
from pandas import Timestamp

In [ ]:
logger = logging.getLogger(__name__)

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger.setLevel(logging.DEBUG)

In [ ]:
def remove_values(values: list, to_remove: list) -> list:
    """
    Removes values from a list based on another list.
  
    Args:
        values: The list from which elements will be removed.
        to_remove: The list containing the values to be removed.
  
    Returns:
        A new list with the values from values_list that are not present in to_remove_list.
    """

    # Use list comprehension for a concise and efficient approach
    return [value for value in values if value not in to_remove]

In [ ]:
satellite_json: os.path = "../data/LightSail-2/cache/normalized_frames.json"

with open(satellite_json, 'r') as file:
    json_stream: str = file.read()

json_data: dict = json.loads(json_stream)

if not json_data:
    logger.error("Json data is empty")
    exit(1)


class Satellite:
    def __init__(self, norad_id: int, name: str, observations_amount: int):
        self.norad_id: int = norad_id
        self.name: str = name
        self.observations_amount: int = observations_amount


metadata: dict = json_data['metadata']

satellite: Satellite = Satellite(
    metadata['satellite_norad'],
    metadata['satellite_name'],
    metadata['total_frames']
)

# Note '...' means there might be more same data

# column_tags look like:
# "column_tags": {
#               "some_variable": "variable",
#               "some_constant": "constant",
#               ... 
#               "some_status": "status"
#             }
solar_and_satellite_parameters: dict = metadata['analysis']['column_tags']

# feature_columns looks like:
# "feature_columns": [
#                 "Fredericksburg A",
#                 "Fredericksburg K 0-3"
#                 ]
solar_parameters: list = metadata['analysis']['feature_columns']

satellite_parameters: list = remove_values(
    list(solar_and_satellite_parameters.keys()), solar_parameters)

# frames look like (fields might contain both satellite and solar parameters):
# "frames": [
#         {
#             "time": "2019-07-02 18:15:08",
#             "measurement": "",
#             "tags": {
#                 "satellite": "",
#                 "decoder": "Lightsail2",
#                 "station": "",
#                 "observer": "",
#                 "source": "",
#                 "version": "1.66.0"
#             },
#             "fields": {
#                 "dest_callsign": {
#                     "value": "N6CP  ",
#                     "unit": null
#                 },
#                 "Fredericksburg A": {
#                     "value": 3.3125,
#                     "unit": "V"
#                 },
#                 ...
#             }
#         },
#         ...
#     ]

frames: list = json_data['frames']

allowed_types: Union = Union[int, float, bool, str]

observation_data: list[dict[str, Timestamp | Timestamp]] = []

for frame in frames:
    observation_time: str = frame['unix_time']
    datetime_time: Timestamp | Timestamp = pd.to_datetime(
        observation_time)

    fields: dict[str, allowed_types] = {key: value['value'] for key, value in
                                        frame['fields'].items()}
    observation_data.append(
        {'unix_time': datetime_time, **fields})  # Unpack fields dictionary

# Create DataFrame
observation_dataframe: pd.DataFrame = pd.DataFrame(observation_data)
observation_dataframe = observation_dataframe.dropna()
observation_dataframe = observation_dataframe.set_index('unix_time')
print(observation_dataframe)